# Illinois Weather


In [13]:
import pandas as pd
import geopandas as gpd
pd.read_csv("../data/processed/processed_rend_lake_data.csv")

,year,month,day,max_wind_gust,avg_wind_speed,avg_wind_dir,sol_rad,max_air_temp,min_air_temp,avg_air_temp,...,avg_rel_hum,avg_dewpt_temp,precip,pot_evapot,max_soiltemp_4in_sod,min_soiltemp_4in_sod,avg_soiltemp_4in_sod,max_soiltemp_8in_sod,min_soiltemp_8in_sod,avg_soiltemp_8in_sod
0,1990,4,18,24.2,7.9,73.3,27.77,57.5,35.7,47.1,...,52.4,29.1,0,0.19,56.3,45.6,50.7,53.7,48.2,51.1
1,1990,4,19,15.2,4.4,122.2,6.87,62.4,43.6,54.1,...,79.9,48,0,0.05,54.1,48,51.1,53.2,49.4,51.3
2,1990,4,20,24.7,7.1,193.6,4.56,67.4,60.7,63.9,...,94.1,62.1,0.01,0.04,57.7,52,54.9,55.9,51.5,53.4
3,1990,4,21,14.4,3.9,350.8,18.29,70.5,54.8,63.5,...,94.1,61.7,0,0.15,65.5,55.4,59.9,60.7,54.2,57
4,1990,4,22,10,2.7,99,24.87,76.2,48.2,63.7,...,82,57.6,0,0.21,68.3,55.1,61.3,62.9,55.6,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11211,2020,12,27,27.4,8.8,208.8,8.7,57.9,36.9,47.7,...,68.7,37.7,0,0.07,39.9,34.2,36.9,39.4,36.4,37.3
11212,2020,12,28,14.8,4.9,254.6,6.2,41.8,26,33.2,...,73.6,25.6,0,0.03,39.5,36.7,38.1,39.6,38.7,39.2
11213,2020,12,29,21,9.2,100.2,3.9,41.7,25.4,32.8,...,74.3,25.4,0,0.03,37.9,35.7,36.8,39.1,37.9,38.4
11214,2020,12,30,23.6,6.1,260.1,1.2,50,32,41.5,...,88.4,38.1,1.08,0.01,42.1,37.9,40.1,41.5,38.8,40.2


In [12]:
# Illinois county
evi_stacked = pd.read_csv("../data/raw/EVI_stacked.csv")


In [32]:
geo_dataframe = gpd.read_file("../data/raw/illinois-counties.geojson")
geo_dataframe["geometry"].iloc[0].centroid.y

39.35625290809104

In [31]:
type(geo_dataframe["geometry"])

geopandas.geoseries.GeoSeries

In [37]:
# Get centroid
def getXY(pt):
    """
    Function returns x and y coordinates of geo pandas centroid
    @param pt:
    @return:
    """
    return pt.x, pt.y
centroidseries = geo_dataframe['geometry'].centroid
x,y = [list(t) for t in zip(*map(getXY, centroidseries))]

<ipython-input-37-548f943b1e4f>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroidseries = geo_dataframe['geometry'].centroid


In [44]:
names = list(geo_dataframe["name"])

In [51]:
# County centroid dataframe
county_centroid_dataframe = pd.DataFrame()
county_centroid_dataframe["county"] = names
county_centroid_dataframe["long"] = x
county_centroid_dataframe["lat"] = y
county_centroid_dataframe

,county,long,lat
0,Greene,-90.390366,39.356253
1,Clinton,-89.422525,38.606437
2,Tazewell,-89.513398,40.507512
3,Fulton,-90.207442,40.472766
4,Adams,-91.188430,39.987842
...,...,...,...
97,Williamson,-88.929957,37.730180
98,Hamilton,-88.539164,38.081574
99,Mcdonough,-90.677893,40.456247
100,Jefferson,-88.924003,38.300533


In [59]:
# Zones
zone1_min_lat = evi_stacked[evi_stacked["county"] == "LIVINGSTON"]["lat"].max()
zone3_max_lat = evi_stacked[evi_stacked["county"] == "CRAWFORD"]["lat"].max()
zone3_max_lat

39.175

In [70]:
zone_1_counties = list(county_centroid_dataframe[county_centroid_dataframe["lat"] >
                                         zone1_min_lat]["county"])
zone_3_counties = list(county_centroid_dataframe[county_centroid_dataframe["lat"] <
                                         zone3_max_lat]["county"])
current_county_zone = zone_1_counties + zone_3_counties

zone_2_counties = list(county_centroid_dataframe[~county_centroid_dataframe["county"].isin
(current_county_zone)
]["county"])

In [77]:
if len(x) > len(set(x)):
   pass #

In [75]:
x = current_county_zone + zone_3_counties + zone_1_counties

In [76]:
x

['Bureau',
 'Henry',
 'Mercer',
 'Lake',
 'Cook',
 'Rock Island',
 'Carroll',
 'Whiteside',
 'Jo Daviess',
 'Ogle',
 'Will',
 'Mchenry',
 'Lasalle',
 'Putnam',
 'Kankakee',
 'Dekalb',
 'Grundy',
 'Boone',
 'Dupage',
 'Kane',
 'Kendall',
 'Stephenson',
 'Lee',
 'Winnebago',
 'Clinton',
 'Gallatin',
 'Johnson',
 'Union',
 'Randolph',
 'Perry',
 'Franklin',
 'Calhoun',
 'Clay',
 'Jersey',
 'Hardin',
 'Jackson',
 'Pope',
 'Madison',
 'St. Clair',
 'Monroe',
 'Crawford',
 'Wabash',
 'Edwards',
 'Washington',
 'Richland',
 'White',
 'Pulaski',
 'Alexander',
 'Saline',
 'Lawrence',
 'Massac',
 'Fayette',
 'Wayne',
 'Jasper',
 'Bond',
 'Effingham',
 'Williamson',
 'Hamilton',
 'Jefferson',
 'Marion',
 'Clinton',
 'Gallatin',
 'Johnson',
 'Union',
 'Randolph',
 'Perry',
 'Franklin',
 'Calhoun',
 'Clay',
 'Jersey',
 'Hardin',
 'Jackson',
 'Pope',
 'Madison',
 'St. Clair',
 'Monroe',
 'Crawford',
 'Wabash',
 'Edwards',
 'Washington',
 'Richland',
 'White',
 'Pulaski',
 'Alexander',
 'Saline',
 'L

## Grouping Data by Zone


In [248]:
era5 = pd.read_csv("../data/raw/era5.csv")
era5["time"] = pd.to_datetime(era5["time"])
era5["year"] = era5["time"].dt.year
era5["month"] = era5["time"].dt.month
era5["day"] = era5["time"].dt.day
era5 = era5[era5["year"] >= 2001]
era5 = era5[(era5["month"] >= 4) & (era5["month"] <= 10)]
era5

,lat,long,time,t2m,county,year,month,day
57213,42.50,-90.5,2001-04-01,275.935944,JO DAVIESS,2001,4,1
57214,42.50,-90.5,2001-04-15,289.480621,JO DAVIESS,2001,4,15
57215,42.50,-90.5,2001-04-28,293.452332,JO DAVIESS,2001,4,28
57216,42.50,-90.5,2001-05-01,295.933533,JO DAVIESS,2001,5,1
57217,42.50,-90.5,2001-05-15,298.438080,JO DAVIESS,2001,5,15
...,...,...,...,...,...,...,...,...
328669,37.25,-89.0,2012-09-15,294.402069,PULASKI,2012,9,15
328670,37.25,-89.0,2012-09-28,295.466919,PULASKI,2012,9,28
328671,37.25,-89.0,2012-10-01,293.211304,PULASKI,2012,10,1
328672,37.25,-89.0,2012-10-15,292.251190,PULASKI,2012,10,15


In [89]:
county_name_list = list()
min_t2m = list()
max_t2m = list()
mean_t2m = list()
era5.groupby("county").mean()


,lat,long,t2m,year,month,day
county,,,,,,
ADAMS,40.000,-91.125,287.551790,2010.000000,6.5,14.666667
ALEXANDER,37.250,-89.250,290.174058,2009.375000,6.5,14.666667
BOND,38.875,-89.500,288.881794,2008.937500,6.5,14.666667
BOONE,42.250,-88.750,284.498382,2009.888889,6.5,14.666667
BROWN,40.000,-90.750,287.558604,2009.500000,6.5,14.666667
...,...,...,...,...,...,...
WHITESIDE,41.750,-89.875,285.678438,2010.000000,6.5,14.666667
WILL,41.500,-88.125,285.720088,2009.117647,6.5,14.666667
WILLIAMSON,37.750,-88.875,290.024419,2010.000000,6.5,14.666667


In [216]:
a = era5[era5["county"] == "BOONE"]
t2m_df_year = pd.DataFrame()
for year in sorted(list(a["year"].unique())):
    b = a[a["year"] == year]

    month_list = list(b["month"].unique())
    monthly_mean_t2m_list = list(b.groupby("month").mean()["t2m"])
    monthly_max_t2m_list = list(b.groupby("month").max()["t2m"])
    monthly_min_t2m_list = list(b.groupby("month").min()["t2m"])

    t2m_df = pd.DataFrame()
    t2m_df["month"] = month_list
    t2m_df["mean"] = monthly_mean_t2m_list
    t2m_df["max"] = monthly_max_t2m_list
    t2m_df["min"] = monthly_min_t2m_list
    t2m_df["year"] = year
    t2m_df_year = t2m_df_year.append(t2m_df)

In [217]:
t2m_df_year

,month,mean,max,min,year
0,4,284.721191,288.360535,277.901306,2001
1,5,292.893361,297.142029,289.013763,2001
2,6,294.788015,300.558502,283.385223,2001
3,7,299.964467,302.304535,296.612518,2001
4,8,300.005168,304.584869,296.614105,2001
...,...,...,...,...,...
2,6,297.277395,298.842041,295.280151,2019
3,7,299.498077,302.484070,296.979584,2019
4,8,296.334178,296.961365,295.617126,2019
5,9,293.951396,296.244629,292.586212,2019


In [218]:
filtered_year_t2m = t2m_df_year[t2m_df_year["year"] == 2001]


In [219]:
columns = t2m_df_year.columns[1:-1]
columns

Index(['mean', 'max', 'min'], dtype='object')

In [220]:
month_number_columns = ["month_" + str(x) for x in list(filtered_year_t2m["month"])]
month_number_mean_columns = [x + "_mean" for x in month_number_columns]
month_number_max_columns = [x + "_max" for x in month_number_columns]
month_number_min_columns = [x + "_min" for x in month_number_columns]

In [221]:
columns_to_filter = ["mean", "max", "min"]

concatenated_dataframe = pd.DataFrame()
for column in columns_to_filter:
    temporary_dataframe = pd.DataFrame(filtered_year_t2m[column]).T
    if column == "mean":
        temporary_dataframe.columns = [month_number_mean_columns]
    elif column == "max":
        temporary_dataframe.columns = [month_number_max_columns]
    else:
        temporary_dataframe.columns = [month_number_min_columns]
    temporary_dataframe.reset_index(inplace=True, drop=True)
    concatenated_dataframe = pd.concat([concatenated_dataframe, temporary_dataframe], axis=1)

In [222]:
concatenated_dataframe.shape
concatenated_dataframe

,month_4_mean,month_5_mean,month_6_mean,month_7_mean,month_8_mean,month_9_mean,month_10_mean,month_4_max,month_5_max,month_6_max,...,month_8_max,month_9_max,month_10_max,month_4_min,month_5_min,month_6_min,month_7_min,month_8_min,month_9_min,month_10_min
0,284.721191,292.893361,294.788015,299.964467,300.005168,288.825755,282.702779,288.360535,297.142029,300.558502,...,304.584869,292.582947,288.78476,277.901306,289.013763,283.385223,296.612518,296.614105,286.72699,276.030762


In [191]:
APRIL_TO_OCTOBER = [4, 5, 6, 7, 8, 9, 10]
MONTH_NUMBER_COLUMNS = ["month_" + str(x) for x in APRIL_TO_OCTOBER]
MONTH_NUMBER_MEAN_COLUMNS = [x + "_mean" for x in MONTH_NUMBER_COLUMNS]
MONTH_NUMBER_MAX_COLUMNS = [x + "_max" for x in MONTH_NUMBER_COLUMNS]
MONTH_NUMBER_MIN_COLUMNS = [x + "_min" for x in MONTH_NUMBER_COLUMNS]
COLUMNS_TO_FILTER = ["mean", "max", "min"]

def make_1_by_21_dataframe(dataframe):
    """

    @param dataframe:
    @return:
    """
    concatenated_dataframe = pd.DataFrame()
    for column in COLUMNS_TO_FILTER:
        temporary_dataframe = pd.DataFrame(dataframe[column]).T
        if column == "mean":
            temporary_dataframe.columns = [MONTH_NUMBER_MEAN_COLUMNS]
        elif column == "max":
            temporary_dataframe.columns = [MONTH_NUMBER_MAX_COLUMNS]
        else:
            temporary_dataframe.columns = [MONTH_NUMBER_MIN_COLUMNS]
        temporary_dataframe.reset_index(inplace=True, drop=True)
        concatenated_dataframe = pd.concat([concatenated_dataframe, temporary_dataframe], axis=1)
    return concatenated_dataframe


In [245]:
import numpy as np
APRIL_TO_OCTOBER = [4, 5, 6, 7, 8, 9, 10]
MONTH_NUMBER_COLUMNS = ["month_" + str(x) for x in APRIL_TO_OCTOBER]
MONTH_NUMBER_MEAN_COLUMNS = [x + "_mean" for x in MONTH_NUMBER_COLUMNS]
MONTH_NUMBER_MAX_COLUMNS = [x + "_max" for x in MONTH_NUMBER_COLUMNS]
MONTH_NUMBER_MIN_COLUMNS = [x + "_min" for x in MONTH_NUMBER_COLUMNS]
COLUMNS_TO_FILTER = ["mean", "max", "min"]
LIST_2001_TO_2019 = [2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
                     2013, 2014, 2015, 2016, 2017, 2018, 2019]

def make_1_by_21_dataframe(dataframe):
    """

    @param dataframe:
    @return:
    """
    concatenated_dataframe = pd.DataFrame()
    for column in COLUMNS_TO_FILTER:
        temporary_dataframe = pd.DataFrame(dataframe[column]).T
        if column == "mean":
            temporary_dataframe.columns = [MONTH_NUMBER_MEAN_COLUMNS]
        elif column == "max":
            temporary_dataframe.columns = [MONTH_NUMBER_MAX_COLUMNS]
        else:
            temporary_dataframe.columns = [MONTH_NUMBER_MIN_COLUMNS]
        temporary_dataframe.reset_index(inplace=True, drop=True)
        concatenated_dataframe = pd.concat([concatenated_dataframe, temporary_dataframe], axis=1)
    return concatenated_dataframe

def create_monthly_metric_list(dataframe, column):
    """

    @param dataframe:
    @return:
    """
    monthly_mean_metric_list = list(dataframe.groupby("month").mean()[column])
    monthly_max_metric_list = list(dataframe.groupby("month").max()[column])
    monthly_min_metric_list = list(dataframe.groupby("month").min()[column])
    return monthly_mean_metric_list, monthly_max_metric_list, monthly_min_metric_list

def create_monthly_metric_dataframe(month_list, monthly_mean_metric_list,
                                    monthly_max_metric_list, monthly_min_metric_list, year):
    """

    @param month_list:
    @param monthly_mean_metric_list:
    @param monthly_max_metric_list:
    @param monthly_min_metric_list:
    @return:
    """
    monthly_metric_dataframe = pd.DataFrame()
    monthly_metric_dataframe["month"] = month_list
    try:
        monthly_metric_dataframe["mean"] = monthly_mean_metric_list
        monthly_metric_dataframe["max"] = monthly_max_metric_list
        monthly_metric_dataframe["min"] = monthly_min_metric_list
    except:
        monthly_metric_dataframe["mean"] = np.nan
        monthly_metric_dataframe["max"] = np.nan
        monthly_metric_dataframe["min"] = np.nan
    monthly_metric_dataframe["year"] = year
    return monthly_metric_dataframe

def get_county_metric_history(dataframe, column):
    """

    @param dataframe: monthly_metric_dataframe
    @return:
    """
    county_metric_history_dataframe = pd.DataFrame()
    for year in LIST_2001_TO_2019:
        try:
            temporary_dataframe = dataframe[dataframe["year"] == year]
            monthly_mean_metric_list, monthly_max_metric_list, monthly_min_metric_list = \
                create_monthly_metric_list(temporary_dataframe, column)
            monthly_metric_dataframe = create_monthly_metric_dataframe(APRIL_TO_OCTOBER,
                                                                       monthly_mean_metric_list,
                                                                       monthly_max_metric_list,
                                                                       monthly_min_metric_list,
                                                                       year)
        except:
            pass
        county_metric_history_dataframe = county_metric_history_dataframe.append\
                (monthly_metric_dataframe)
    return county_metric_history_dataframe

def create_yearly_1_by_21_dataframe(dataframe):
    """

    @param dataframe: county_metric_history_dataframe
    @return:
    """
    yearly_1_by_21_dataframe = pd.DataFrame()
    for year in LIST_2001_TO_2019:
        temporary_dataframe = dataframe[dataframe["year"] == year]
        concatenated_dataframe = make_1_by_21_dataframe(temporary_dataframe)
        concatenated_dataframe["year"] = year
        yearly_1_by_21_dataframe = yearly_1_by_21_dataframe.append(concatenated_dataframe)
    return yearly_1_by_21_dataframe

def create_county_yearly_1_by_21_dataframe(dataframe, column, metric):
    """

    @param dataframe:
    @param column: column for for-loop
    @return:
    """
    county_yearly_1_by_21_dataframe = pd.DataFrame()
    for n in list(dataframe[column].unique()):
        temporary_dataframe = dataframe[dataframe[column] == n]
        county_metric_history_dataframe = get_county_metric_history(temporary_dataframe, metric)
        yearly_1_by_21_dataframe = create_yearly_1_by_21_dataframe(county_metric_history_dataframe)
        yearly_1_by_21_dataframe[column] = n
        county_yearly_1_by_21_dataframe = county_yearly_1_by_21_dataframe.append\
            (yearly_1_by_21_dataframe)
    return county_yearly_1_by_21_dataframe

In [288]:
country_yearly_evi_dataframe = create_county_yearly_1_by_21_dataframe(era5, "county", "t2m")
country_yearly_evi_dataframe

,month_4_mean,month_5_mean,month_6_mean,month_7_mean,month_8_mean,month_9_mean,month_10_mean,month_4_max,month_5_max,month_6_max,...,month_10_max,month_4_min,month_5_min,month_6_min,month_7_min,month_8_min,month_9_min,month_10_min,year,county
0,285.832381,294.297239,293.872882,298.844855,299.906551,289.426021,282.642635,293.452332,298.438080,300.386566,...,289.680695,275.404816,288.978424,282.576904,296.891327,295.518127,287.005829,274.866486,2001,JO DAVIESS
0,284.136558,290.352519,296.349707,302.778060,299.074068,292.490265,286.029022,295.027985,295.771240,299.252808,...,299.134705,277.517242,285.778870,290.865845,300.266052,295.898193,286.046448,275.833893,2002,JO DAVIESS
0,292.760720,288.676404,295.585685,299.745477,299.708085,288.692049,281.721918,299.467346,298.147186,299.094543,...,286.170166,283.038910,282.178375,289.351257,298.785187,297.303528,284.964111,277.569458,2003,JO DAVIESS
0,285.571021,286.626823,294.700321,298.129020,296.767053,296.059888,286.338981,291.793091,295.898712,299.097107,...,291.315521,280.986450,280.535614,291.679688,297.209808,293.222198,292.903168,281.603271,2004,JO DAVIESS
0,286.380654,284.454185,298.727940,300.070789,298.824015,294.486420,288.417019,290.985443,288.865784,305.097412,...,292.419128,282.666321,281.451141,294.782196,295.401764,296.477722,292.492889,280.504242,2005,JO DAVIESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,291.925323,294.587240,297.188934,302.683462,300.422506,298.161530,290.729116,295.180481,296.006439,301.812775,...,293.037537,290.070496,291.803619,291.162018,301.055267,297.327972,296.321289,287.906555,2015,MASSAC
0,293.590322,293.022644,301.426737,300.766276,301.256673,298.838114,289.392069,293.774078,298.108307,302.323883,...,290.205566,293.470154,287.450226,300.302338,299.573120,298.922394,295.155823,288.429504,2016,MASSAC
0,290.812693,297.144124,301.874196,301.217244,299.877248,298.016917,290.725576,299.117096,299.754608,304.628876,...,299.339325,284.558044,294.043365,299.800140,299.883026,298.550598,297.544250,278.540833,2017,MASSAC
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,MASSAC


In [289]:
country_yearly_evi_dataframe.columns=country_yearly_evi_dataframe.columns.get_level_values(0).map\
    (lambda x : "t2m_" + x if x !='year' and x!='county' else x)
country_yearly_evi_dataframe

,t2m_month_4_mean,t2m_month_5_mean,t2m_month_6_mean,t2m_month_7_mean,t2m_month_8_mean,t2m_month_9_mean,t2m_month_10_mean,t2m_month_4_max,t2m_month_5_max,t2m_month_6_max,...,t2m_month_10_max,t2m_month_4_min,t2m_month_5_min,t2m_month_6_min,t2m_month_7_min,t2m_month_8_min,t2m_month_9_min,t2m_month_10_min,year,county
0,285.832381,294.297239,293.872882,298.844855,299.906551,289.426021,282.642635,293.452332,298.438080,300.386566,...,289.680695,275.404816,288.978424,282.576904,296.891327,295.518127,287.005829,274.866486,2001,JO DAVIESS
0,284.136558,290.352519,296.349707,302.778060,299.074068,292.490265,286.029022,295.027985,295.771240,299.252808,...,299.134705,277.517242,285.778870,290.865845,300.266052,295.898193,286.046448,275.833893,2002,JO DAVIESS
0,292.760720,288.676404,295.585685,299.745477,299.708085,288.692049,281.721918,299.467346,298.147186,299.094543,...,286.170166,283.038910,282.178375,289.351257,298.785187,297.303528,284.964111,277.569458,2003,JO DAVIESS
0,285.571021,286.626823,294.700321,298.129020,296.767053,296.059888,286.338981,291.793091,295.898712,299.097107,...,291.315521,280.986450,280.535614,291.679688,297.209808,293.222198,292.903168,281.603271,2004,JO DAVIESS
0,286.380654,284.454185,298.727940,300.070789,298.824015,294.486420,288.417019,290.985443,288.865784,305.097412,...,292.419128,282.666321,281.451141,294.782196,295.401764,296.477722,292.492889,280.504242,2005,JO DAVIESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,291.925323,294.587240,297.188934,302.683462,300.422506,298.161530,290.729116,295.180481,296.006439,301.812775,...,293.037537,290.070496,291.803619,291.162018,301.055267,297.327972,296.321289,287.906555,2015,MASSAC
0,293.590322,293.022644,301.426737,300.766276,301.256673,298.838114,289.392069,293.774078,298.108307,302.323883,...,290.205566,293.470154,287.450226,300.302338,299.573120,298.922394,295.155823,288.429504,2016,MASSAC
0,290.812693,297.144124,301.874196,301.217244,299.877248,298.016917,290.725576,299.117096,299.754608,304.628876,...,299.339325,284.558044,294.043365,299.800140,299.883026,298.550598,297.544250,278.540833,2017,MASSAC
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,MASSAC


In [290]:
country_yearly_evi_dataframe.to_csv("../data/interim/country_yearly_t2m_1_by_n.csv")